In [ ]:
# Cài đặt PyTorch 2.8.0 + torchvision + torchaudio tương thích
!pip install torch==2.8.0+cu128 torchvision==0.23.0+cu128 torchaudio==2.8.0+cu128 --extra-index-url https://download.pytorch.org/whl/cu128

# Cài transformers và accelerate tương thích
print("Installing transformers and dependencies...")
!pip install transformers==4.52.4 accelerate==1.7.0 datasets>=3.6.0

print("Installing other dependencies...")
!pip install librosa soundfile audiomentations==0.35.0 wandb safetensors

import torch
print("\n" + "="*60)
print("✓ Installation complete!")
print("="*60)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")

import torchvision
print(f"Torchvision version: {torchvision.__version__}")

import transformers
print(f"Transformers version: {transformers.__version__}")

In [1]:
print("Installing dependencies...")
!pip install -q transformers==4.44.0 accelerate==0.33.0 datasets==2.21.0
!pip install -q librosa soundfile audiomentations==0.35.0 wandb safetensors

Installing dependencies...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 77.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 94.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8

In [2]:
# 2. IMPORTS
import os
import pandas as pd
import numpy as np
import torch
import librosa
import json
import wandb
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from transformers import (
    Wav2Vec2FeatureExtractor,
    AutoModel,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
from torch.utils.data import Dataset, ConcatDataset
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, Gain
from datasets import load_dataset
import random
import warnings
import gc
warnings.filterwarnings('ignore')

print("All libraries imported successfully")

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

All libraries imported successfully


In [3]:
# 3. CONFIGURATION
class Config:
    """Optimized configuration for combined datasets"""
    
    # ViSpeech Paths
    VISPEECH_ROOT = Path('/kaggle/input/vispeech')
    TRAIN_META = VISPEECH_ROOT / 'metadata/trainset.csv'
    CLEAN_TEST_META = VISPEECH_ROOT / 'metadata/clean_testset.csv'
    NOISY_TEST_META = VISPEECH_ROOT / 'metadata/noisy_testset.csv'
    TRAIN_AUDIO = VISPEECH_ROOT / 'trainset'
    CLEAN_TEST_AUDIO = VISPEECH_ROOT / 'clean_testset'
    NOISY_TEST_AUDIO = VISPEECH_ROOT / 'noisy_testset'
    
    # ViMD Path
    VIMD_PATH = '/kaggle/input/vimd-dataset'
    
    # Model
    MODEL_NAME = "microsoft/wavlm-base-plus"
    
    # Audio Processing
    SAMPLING_RATE = 16000
    MAX_DURATION = 5  
    
    # Training
    BATCH_SIZE = 32       
    LEARNING_RATE = 5e-5    
    NUM_EPOCHS = 5      
    WARMUP_RATIO = 0.12  
    WEIGHT_DECAY = 0.012  
    GRADIENT_CLIP = 1.0
    
    # Data Augmentation
    AUGMENT_PROB = 0.8
    
    # Loss weighting
    DIALECT_LOSS_WEIGHT = 3.0 
    
    # Label mappings (unified for both datasets)
    GENDER_MAP = {'Male': 0, 'Female': 1, 0: 0, 1: 1}  # Support both string and int
    DIALECT_MAP = {'North': 0, 'Central': 1, 'South': 2}
    
    # ViMD specific: region name to dialect mapping
    REGION_TO_DIALECT = {'North': 0, 'Central': 1, 'South': 2}
    
    # Wandb tracking
    WANDB_PROJECT = 'vietnamese-speaker-profiling'
    WANDB_API_KEY = 'f05e29c3466ec288e97041e0e3d541c4087096a6'
    
    # Logging (save to file as backup)
    LOG_DIR = '/kaggle/working/logs'
    
    # Output
    OUTPUT_DIR = '/kaggle/working/speaker-profiling-combined'
    
    # Reproducibility
    SEED = 42

config = Config()

# Create necessary directories
os.makedirs(config.LOG_DIR, exist_ok=True)
os.makedirs(config.OUTPUT_DIR, exist_ok=True)

# Set seeds
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(config.SEED)
# Setup Wandb
os.environ['WANDB_API_KEY'] = config.WANDB_API_KEY
wandb.login(key=config.WANDB_API_KEY)

print("\n" + "="*80)
print("CONFIGURATION")
print("="*80)
print(f"Model:              {config.MODEL_NAME}")
print(f"Sampling Rate:      {config.SAMPLING_RATE} Hz")
print(f"Max Duration:       {config.MAX_DURATION}s")
print(f"Batch Size:         {config.BATCH_SIZE}")
print(f"Learning Rate:      {config.LEARNING_RATE}")
print(f"Epochs:             {config.NUM_EPOCHS}")
print("="*80)
print(f"Log Directory:      {config.LOG_DIR}")
print(f"Wandb Project:      {config.WANDB_PROJECT}")
print(f"Dialect Loss Weight:{config.DIALECT_LOSS_WEIGHT}x")
print(f"Augmentation Prob:  {config.AUGMENT_PROB}")
print("="*80)
print(f"✓ Directories created successfully")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vuthanhlam848 (vuthanhlam848-vnpost) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



CONFIGURATION
Model:              microsoft/wavlm-base-plus
Sampling Rate:      16000 Hz
Max Duration:       5s
Batch Size:         32
Learning Rate:      5e-05
Epochs:             5
Log Directory:      /kaggle/working/logs
Wandb Project:      vietnamese-speaker-profiling
Dialect Loss Weight:3.0x
Augmentation Prob:  0.8
✓ Directories created successfully


In [4]:
# 4. DATA LOADING (Memory Optimized - Không dùng torchcodec)

def load_vispeech_data():
    """Load ViSpeech dataset"""
    print("\nLoading ViSpeech dataset...")
    
    # Load metadata
    train_df = pd.read_csv(config.TRAIN_META)
    clean_test_df = pd.read_csv(config.CLEAN_TEST_META)
    noisy_test_df = pd.read_csv(config.NOISY_TEST_META)
    
    # Combine clean + noisy test into one test set
    test_df = pd.concat([
        clean_test_df.assign(audio_dir=str(config.CLEAN_TEST_AUDIO)),
        noisy_test_df.assign(audio_dir=str(config.NOISY_TEST_AUDIO))
    ], ignore_index=True)
    
    # Add audio_dir to train
    train_df['audio_dir'] = str(config.TRAIN_AUDIO)
    
    # Encode labels
    for df in [train_df, test_df]:
        df['gender_label'] = df['gender'].map(config.GENDER_MAP)
        df['dialect_label'] = df['dialect'].map(config.DIALECT_MAP)
    
    # Speaker-based split
    unique_speakers = train_df['speaker'].unique()
    train_speakers, val_speakers = train_test_split(
        unique_speakers,
        test_size=0.15,
        random_state=config.SEED,
        shuffle=True
    )
    
    train_data = train_df[train_df['speaker'].isin(train_speakers)].reset_index(drop=True)
    val_data = train_df[train_df['speaker'].isin(val_speakers)].reset_index(drop=True)
    
    print(f"  ViSpeech Train:      {len(train_data):,} samples ({len(train_speakers)} speakers)")
    print(f"  ViSpeech Validation: {len(val_data):,} samples ({len(val_speakers)} speakers)")
    print(f"  ViSpeech Test:       {len(test_df):,} samples (clean + noisy combined)")
    
    return train_data, val_data, test_df


def load_vimd_data():
    """Load ViMD dataset - KHÔNG dùng Audio feature để tránh lỗi torchcodec"""
    print("\nLoading ViMD dataset...")
    
    try:
        # Load dataset WITHOUT audio decoding (chỉ lấy metadata)
        # Audio sẽ được load trực tiếp trong Dataset class bằng librosa
        ds = load_dataset(
            config.VIMD_PATH, 
            keep_in_memory=False
        )
        
        # Check available splits
        available_splits = list(ds.keys())
        print(f"  Available splits: {available_splits}")
        
        # Handle both 'valid' and 'validation' keys
        val_key = None
        if 'validation' in available_splits:
            val_key = 'validation'
        elif 'valid' in available_splits:
            val_key = 'valid'
        else:
            raise KeyError(f"No validation split found. Available: {available_splits}")
        
        print(f"  ViMD Train:          {len(ds['train']):,} samples")
        print(f"  ViMD Validation:     {len(ds[val_key]):,} samples")
        print(f"  ViMD Test:           {len(ds['test']):,} samples")
        
        # Normalize split name to 'valid'
        if val_key == 'validation':
            ds['valid'] = ds['validation']
            del ds['validation']
            gc.collect()
        
        # Kiểm tra columns
        print(f"  Available columns: {ds['train'].column_names}")
        
        # Tìm column chứa audio path
        audio_col = None
        for col in ['audio', 'path', 'file', 'audio_path', 'filename']:
            if col in ds['train'].column_names:
                audio_col = col
                break
        
        if audio_col:
            print(f"  Audio column: '{audio_col}'")
            # Kiểm tra cấu trúc audio column
            sample = ds['train'][0]
            if audio_col in sample:
                audio_val = sample[audio_col]
                if isinstance(audio_val, dict) and 'path' in audio_val:
                    print(f"  Audio path example: {audio_val['path']}")
                elif isinstance(audio_val, str):
                    print(f"  Audio path example: {audio_val}")
        
        return ds
    
    except Exception as e:
        print(f"Error loading ViMD dataset: {e}")
        raise


# Load both datasets
vispeech_train, vispeech_val, vispeech_test = load_vispeech_data()
vimd_dataset = load_vimd_data()

print("\n" + "="*80)
print("TOTAL DATA")
print("="*80)
print(f"Train:      {len(vispeech_train) + len(vimd_dataset['train']):,} samples")
print(f"Validation: {len(vispeech_val) + len(vimd_dataset['valid']):,} samples")
print(f"Test:       {len(vispeech_test) + len(vimd_dataset['test']):,} samples")
print("="*80)

# Clean up
gc.collect()


Loading ViSpeech dataset...
  ViSpeech Train:      7,137 samples (263 speakers)
  ViSpeech Validation: 1,029 samples (47 speakers)
  ViSpeech Test:       2,520 samples (clean + noisy combined)

Loading ViMD dataset...


Resolving data files:   0%|          | 0/87 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/87 [00:00<?, ?it/s]

  Available splits: ['train', 'validation', 'test']
  ViMD Train:          15,023 samples
  ViMD Validation:     1,900 samples
  ViMD Test:           2,026 samples
  Available columns: ['region', 'province_code', 'province_name', 'filename', 'text', 'speakerID', 'gender', 'audio']
  Audio column: 'audio'
  Audio path example: 11_0001.wav

TOTAL DATA
Train:      22,160 samples
Validation: 2,929 samples
Test:       4,546 samples


67788

In [5]:
# TEST: Kiểm tra cấu trúc dataset ViMD
print("Kiểm tra cấu trúc ViMD dataset...")

try:
    # Get one sample from ViMD train
    sample = vimd_dataset['train'][0]
    
    print(f"\n1. Dataset columns: {vimd_dataset['train'].column_names}")
    print(f"2. Sample keys: {sample.keys()}")
    
    # Kiểm tra audio column
    if 'audio' in sample:
        audio_data = sample['audio']
        print(f"\n3. Audio type: {type(audio_data)}")
        
        if isinstance(audio_data, dict):
            print(f"   Audio keys: {audio_data.keys()}")
            if 'path' in audio_data:
                print(f"   Audio path: {audio_data['path']}")
            if 'bytes' in audio_data:
                print(f"   Audio bytes length: {len(audio_data['bytes']) if audio_data['bytes'] else 0}")
        elif isinstance(audio_data, str):
            print(f"   Audio path: {audio_data}")
    
    # Check labels
    print(f"\n4. Labels:")
    print(f"   Region (dialect): {sample.get('region', 'N/A')}")
    print(f"   Gender: {sample.get('gender', 'N/A')}")
    
    print(f"\n✓ Dataset structure verified!")
    
except Exception as e:
    import traceback
    print(f"✗ Error:")
    traceback.print_exc()

Kiểm tra cấu trúc ViMD dataset...

1. Dataset columns: ['region', 'province_code', 'province_name', 'filename', 'text', 'speakerID', 'gender', 'audio']
2. Sample keys: dict_keys(['region', 'province_code', 'province_name', 'filename', 'text', 'speakerID', 'gender', 'audio'])

3. Audio type: <class 'dict'>
   Audio keys: dict_keys(['path', 'array', 'sampling_rate'])
   Audio path: 11_0001.wav

4. Labels:
   Region (dialect): North
   Gender: 1

✓ Dataset structure verified!


In [6]:
# 5. AUDIO AUGMENTATION

class AudioAugmentation:
    """Audio augmentation pipeline"""
    
    def __init__(self, sampling_rate=16000, augment_prob=0.8):
        self.sampling_rate = sampling_rate
        self.augment_prob = augment_prob
        
        self.augment = Compose([
            AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
            TimeStretch(min_rate=0.8, max_rate=1.2, leave_length_unchanged=False, p=0.5),
            PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
            Shift(min_shift=-0.5, max_shift=0.5, p=0.3),
            Gain(min_gain_db=-12, max_gain_db=12, p=0.5),
        ])
    
    def __call__(self, audio):
        if random.random() < self.augment_prob:
            return self.augment(samples=audio, sample_rate=self.sampling_rate)
        return audio

In [7]:
# 6. DATASET CLASSES (Sử dụng librosa để load audio, không dùng torchcodec)
import soundfile as sf
import io

class ViSpeechDataset(Dataset):
    """Dataset for ViSpeech (CSV-based)"""
    
    def __init__(self, dataframe, feature_extractor,
                 sampling_rate=16000, max_duration=5, is_training=True):
        self.df = dataframe.reset_index(drop=True)
        self.feature_extractor = feature_extractor
        self.sampling_rate = sampling_rate
        self.max_length = int(sampling_rate * max_duration)
        self.is_training = is_training
        
        if is_training:
            self.augmentation = AudioAugmentation(sampling_rate, augment_prob=config.AUGMENT_PROB)
        else:
            self.augmentation = None
    
    def __len__(self):
        return len(self.df)
    
    def load_audio(self, audio_name, audio_dir):
        audio_path = Path(audio_dir) / audio_name
        
        try:
            audio, sr = librosa.load(audio_path, sr=self.sampling_rate, mono=True)
            audio, _ = librosa.effects.trim(audio, top_db=20)
            
            if self.is_training and self.augmentation is not None:
                audio = self.augmentation(audio)
            
            audio = audio / (np.max(np.abs(audio)) + 1e-8)
            
            if len(audio) < self.max_length:
                audio = np.pad(audio, (0, self.max_length - len(audio)))
            else:
                if self.is_training:
                    start = np.random.randint(0, len(audio) - self.max_length + 1)
                else:
                    start = (len(audio) - self.max_length) // 2
                audio = audio[start:start + self.max_length]
            
            return audio
            
        except Exception as e:
            print(f"Error loading {audio_path}: {e}")
            return np.zeros(self.max_length)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        audio = self.load_audio(row['audio_name'], row['audio_dir'])
        
        inputs = self.feature_extractor(
            audio,
            sampling_rate=self.sampling_rate,
            return_tensors="pt",
            padding=True
        )
        
        return {
            'input_values': inputs.input_values.squeeze(0),
            'gender_labels': torch.tensor(row['gender_label'], dtype=torch.long),
            'dialect_labels': torch.tensor(row['dialect_label'], dtype=torch.long)
        }


class ViMDDataset(Dataset):
    """Dataset for ViMD (HuggingFace format) - Load audio bằng librosa/soundfile"""
    
    def __init__(self, hf_dataset, feature_extractor,
                 sampling_rate=16000, max_duration=5, is_training=True):
        self.dataset = hf_dataset
        self.feature_extractor = feature_extractor
        self.sampling_rate = sampling_rate
        self.max_length = int(sampling_rate * max_duration)
        self.is_training = is_training
        
        if is_training:
            self.augmentation = AudioAugmentation(sampling_rate, augment_prob=config.AUGMENT_PROB)
        else:
            self.augmentation = None
    
    def __len__(self):
        return len(self.dataset)
    
    def load_audio_from_hf(self, audio_data):
        """Load audio từ HuggingFace dataset bằng librosa/soundfile (không dùng torchcodec)"""
        try:
            audio = None
            sr = None
            
            # Case 1: audio_data là dict với 'path' hoặc 'bytes'
            if isinstance(audio_data, dict):
                # Thử load từ path trước
                if 'path' in audio_data and audio_data['path']:
                    try:
                        audio, sr = librosa.load(audio_data['path'], sr=self.sampling_rate, mono=True)
                    except Exception:
                        pass
                
                # Nếu không có path, thử load từ bytes
                if audio is None and 'bytes' in audio_data and audio_data['bytes']:
                    try:
                        audio_bytes = io.BytesIO(audio_data['bytes'])
                        audio, sr = sf.read(audio_bytes)
                        if len(audio.shape) > 1:
                            audio = np.mean(audio, axis=1)
                        if sr != self.sampling_rate:
                            audio = librosa.resample(audio, orig_sr=sr, target_sr=self.sampling_rate)
                    except Exception:
                        pass
                
                # Nếu có array (đã decode sẵn)
                if audio is None and 'array' in audio_data:
                    try:
                        audio = np.array(audio_data['array'], dtype=np.float32)
                        sr = audio_data.get('sampling_rate', self.sampling_rate)
                        if sr != self.sampling_rate:
                            audio = librosa.resample(audio, orig_sr=sr, target_sr=self.sampling_rate)
                    except Exception:
                        pass
            
            # Case 2: audio_data là string (path)
            elif isinstance(audio_data, str):
                audio, sr = librosa.load(audio_data, sr=self.sampling_rate, mono=True)
            
            # Validation
            if audio is None or len(audio) == 0:
                return np.zeros(self.max_length, dtype=np.float32)
            
            # Convert to mono if stereo
            if len(audio.shape) > 1:
                audio = np.mean(audio, axis=1)
            
            # Trim silence
            if len(audio) > 100:
                try:
                    audio, _ = librosa.effects.trim(audio, top_db=20)
                except Exception:
                    pass
            
            # Apply augmentation
            if self.is_training and self.augmentation is not None and len(audio) > 100:
                try:
                    audio = self.augmentation(audio)
                except Exception:
                    pass
            
            # Normalize
            if len(audio) > 0:
                max_val = np.max(np.abs(audio))
                if max_val > 1e-8:
                    audio = audio / max_val
                else:
                    return np.zeros(self.max_length, dtype=np.float32)
            
            # Pad or crop
            if len(audio) < self.max_length:
                audio = np.pad(audio, (0, self.max_length - len(audio)))
            else:
                if self.is_training:
                    start = np.random.randint(0, max(1, len(audio) - self.max_length))
                else:
                    start = max(0, (len(audio) - self.max_length) // 2)
                audio = audio[start:start + self.max_length]
            
            return audio.astype(np.float32)
            
        except Exception as e:
            # Không in lỗi để tránh spam console
            return np.zeros(self.max_length, dtype=np.float32)
    
    def __getitem__(self, idx):
        try:
            item = self.dataset[idx]
            
            # Load audio bằng librosa/soundfile
            audio = self.load_audio_from_hf(item.get('audio'))
            
            # Extract features
            inputs = self.feature_extractor(
                audio,
                sampling_rate=self.sampling_rate,
                return_tensors="pt",
                padding=True
            )
            
            # Map labels
            gender_label = item.get('gender', 0)
            region = item.get('region', 'North')
            dialect_label = config.REGION_TO_DIALECT.get(region, 0)
            
            return {
                'input_values': inputs.input_values.squeeze(0),
                'gender_labels': torch.tensor(gender_label, dtype=torch.long),
                'dialect_labels': torch.tensor(dialect_label, dtype=torch.long)
            }
            
        except Exception as e:
            # Return dummy data to prevent crash
            dummy_audio = np.zeros(self.max_length, dtype=np.float32)
            inputs = self.feature_extractor(
                dummy_audio,
                sampling_rate=self.sampling_rate,
                return_tensors="pt",
                padding=True
            )
            return {
                'input_values': inputs.input_values.squeeze(0),
                'gender_labels': torch.tensor(0, dtype=torch.long),
                'dialect_labels': torch.tensor(0, dtype=torch.long)
            }

In [8]:
# 7. MULTI-TASK MODEL

class MultiTaskSpeakerModel(torch.nn.Module):
    def __init__(self, model_name, num_genders=2, num_dialects=3,
                 freeze_encoder=False, dropout=0.1):
        super().__init__()
        
        self.wavlm = AutoModel.from_pretrained(model_name)
        
        if freeze_encoder:
            for param in self.wavlm.parameters():
                param.requires_grad = False
            print("  Encoder FROZEN")
        
        hidden_size = self.wavlm.config.hidden_size
        self.dropout = torch.nn.Dropout(dropout)
        self.gender_head = torch.nn.Linear(hidden_size, num_genders)
        self.dialect_head = torch.nn.Linear(hidden_size, num_dialects)
        
    def forward(self, input_values, gender_labels=None, dialect_labels=None):
        outputs = self.wavlm(input_values)
        hidden_states = outputs.last_hidden_state
        pooled = torch.mean(hidden_states, dim=1)
        pooled = self.dropout(pooled)
        
        gender_logits = self.gender_head(pooled)
        dialect_logits = self.dialect_head(pooled)
        
        loss = None
        if gender_labels is not None and dialect_labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            gender_loss = loss_fct(gender_logits, gender_labels)
            dialect_loss = loss_fct(dialect_logits, dialect_labels)
            loss = gender_loss + config.DIALECT_LOSS_WEIGHT * dialect_loss
        
        return {
            'loss': loss,
            'gender_logits': gender_logits,
            'dialect_logits': dialect_logits
        }

In [9]:
# 8. CUSTOM TRAINER

class MultiTaskTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        gender_labels = inputs.pop("gender_labels")
        dialect_labels = inputs.pop("dialect_labels")
        
        outputs = model(
            input_values=inputs["input_values"],
            gender_labels=gender_labels,
            dialect_labels=dialect_labels
        )
        
        loss = outputs["loss"]
        return (loss, outputs) if return_outputs else loss
    
    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        gender_labels = inputs.pop("gender_labels")
        dialect_labels = inputs.pop("dialect_labels")
        
        with torch.no_grad():
            outputs = model(
                input_values=inputs["input_values"],
                gender_labels=gender_labels,
                dialect_labels=dialect_labels
            )
            loss = outputs["loss"]
        
        return (
            loss,
            (outputs["gender_logits"], outputs["dialect_logits"]),
            (gender_labels, dialect_labels)
        )

In [10]:
# 9. METRICS

def compute_metrics(pred):
    gender_logits, dialect_logits = pred.predictions
    gender_labels, dialect_labels = pred.label_ids
    
    gender_preds = np.argmax(gender_logits, axis=-1)
    dialect_preds = np.argmax(dialect_logits, axis=-1)
    
    gender_acc = accuracy_score(gender_labels, gender_preds)
    gender_f1 = f1_score(gender_labels, gender_preds, average='weighted')
    dialect_acc = accuracy_score(dialect_labels, dialect_preds)
    dialect_f1 = f1_score(dialect_labels, dialect_preds, average='weighted')
    
    return {
        'gender_acc': gender_acc,
        'gender_f1': gender_f1,
        'dialect_acc': dialect_acc,
        'dialect_f1': dialect_f1,
        'combined_f1': (gender_f1 + dialect_f1) / 2
    }

In [11]:
# 10. INITIALIZE MODEL & DATASETS

print("\n" + "="*80)
print("INITIALIZING")
print("="*80)

# Feature extractor
print("\nLoading feature extractor...")
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(config.MODEL_NAME)
print("  Feature extractor loaded")

# Create ViSpeech datasets
print("\nCreating ViSpeech datasets...")
vispeech_train_dataset = ViSpeechDataset(
    vispeech_train, feature_extractor,
    sampling_rate=config.SAMPLING_RATE,
    max_duration=config.MAX_DURATION,
    is_training=True
)
vispeech_val_dataset = ViSpeechDataset(
    vispeech_val, feature_extractor,
    sampling_rate=config.SAMPLING_RATE,
    max_duration=config.MAX_DURATION,
    is_training=False
)
vispeech_test_dataset = ViSpeechDataset(
    vispeech_test, feature_extractor,
    sampling_rate=config.SAMPLING_RATE,
    max_duration=config.MAX_DURATION,
    is_training=False
)
print(f"  ViSpeech: {len(vispeech_train_dataset)} train, {len(vispeech_val_dataset)} val, {len(vispeech_test_dataset)} test")

# Create ViMD datasets
print("\nCreating ViMD datasets...")
vimd_train_dataset = ViMDDataset(
    vimd_dataset['train'], feature_extractor,
    sampling_rate=config.SAMPLING_RATE,
    max_duration=config.MAX_DURATION,
    is_training=True
)
vimd_val_dataset = ViMDDataset(
    vimd_dataset['valid'], feature_extractor,
    sampling_rate=config.SAMPLING_RATE,
    max_duration=config.MAX_DURATION,
    is_training=False
)
vimd_test_dataset = ViMDDataset(
    vimd_dataset['test'], feature_extractor,
    sampling_rate=config.SAMPLING_RATE,
    max_duration=config.MAX_DURATION,
    is_training=False
)
print(f"  ViMD: {len(vimd_train_dataset)} train, {len(vimd_val_dataset)} val, {len(vimd_test_dataset)} test")

# Combine datasets
print("\nCombining datasets...")
combined_train_dataset = ConcatDataset([vispeech_train_dataset, vimd_train_dataset])
combined_val_dataset = ConcatDataset([vispeech_val_dataset, vimd_val_dataset])
combined_test_dataset = ConcatDataset([vispeech_test_dataset, vimd_test_dataset])

print(f"  Combined Train:      {len(combined_train_dataset):,} samples")
print(f"  Combined Validation: {len(combined_val_dataset):,} samples")
print(f"  Combined Test:       {len(combined_test_dataset):,} samples")

# Model
print("\nLoading model...")
model = MultiTaskSpeakerModel(
    config.MODEL_NAME,
    num_genders=2,
    num_dialects=3,
    freeze_encoder=False,
    dropout=0.1
)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"  Model: {config.MODEL_NAME}")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")

# Check GPU availability
if torch.cuda.is_available():
    print(f"  Device: GPU - {torch.cuda.get_device_name(0)}")
    print(f"  CUDA Version: {torch.version.cuda}")
else:
    print(f"  Device: CPU (no GPU available)")

print("\n" + "="*80)

# Clean up memory
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()


INITIALIZING

Loading feature extractor...


preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

  Feature extractor loaded

Creating ViSpeech datasets...
  ViSpeech: 7137 train, 1029 val, 2520 test

Creating ViMD datasets...
  ViMD: 15023 train, 1900 val, 2026 test

Combining datasets...
  Combined Train:      22,160 samples
  Combined Validation: 2,929 samples
  Combined Test:       4,546 samples

Loading model...


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/wavlm-base-plus were not used when initializing WavLMModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing WavLMModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing WavLMModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of WavLMModel were not initialized from the model checkpoint at microsoft/wavlm-base-plus and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictio

  Model: microsoft/wavlm-base-plus
  Total parameters: 94,385,781
  Trainable parameters: 94,385,781
  Device: GPU - Tesla P100-PCIE-16GB
  CUDA Version: 12.4



In [12]:
# 11. TRAINING ARGUMENTS

training_args = TrainingArguments(
    output_dir=config.OUTPUT_DIR,
    eval_strategy='epoch',
    save_strategy='epoch',
    
    # Optimization
    learning_rate=config.LEARNING_RATE,
    per_device_train_batch_size=config.BATCH_SIZE,
    per_device_eval_batch_size=config.BATCH_SIZE,
    num_train_epochs=config.NUM_EPOCHS,
    weight_decay=config.WEIGHT_DECAY,
    warmup_ratio=config.WARMUP_RATIO,
    max_grad_norm=config.GRADIENT_CLIP,
    
    # Scheduler
    lr_scheduler_type='linear',
    
    # Best model selection
    load_best_model_at_end=True,
    metric_for_best_model='dialect_acc',
    greater_is_better=True,
    save_total_limit=3,
    
    # Performance
    fp16=True,  # Enable mixed precision training for faster GPU training
    dataloader_num_workers=0,  # Fix multiprocessing issue with HuggingFace datasets
    
    # Logging
    logging_steps=50,
    logging_first_step=True,
    report_to='wandb',  # Enable Wandb logging
    
    # Misc
    remove_unused_columns=False,
    seed=config.SEED,
)

early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.0025
)

print("✓ Training arguments configured")
print(f"  FP16: {training_args.fp16}")
print(f"  Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

✓ Training arguments configured
  FP16: True
  Device: GPU


In [13]:
# 12. START TRAINING

print("\n" + "="*80)
print("TRAINING")
print("="*80)

# Ensure log directory exists
os.makedirs(config.LOG_DIR, exist_ok=True)

# Initialize Wandb run
wandb.init(
    project=config.WANDB_PROJECT,
    name=f"combined-training-{config.SEED}",
    config={
        'model_name': config.MODEL_NAME,
        'sampling_rate': config.SAMPLING_RATE,
        'max_duration': config.MAX_DURATION,
        'batch_size': config.BATCH_SIZE,
        'learning_rate': config.LEARNING_RATE,
        'num_epochs': config.NUM_EPOCHS,
        'warmup_ratio': config.WARMUP_RATIO,
        'weight_decay': config.WEIGHT_DECAY,
        'augment_prob': config.AUGMENT_PROB,
        'dialect_loss_weight': config.DIALECT_LOSS_WEIGHT,
        'train_samples': len(combined_train_dataset),
        'val_samples': len(combined_val_dataset),
        'test_samples': len(combined_test_dataset),
        'vispeech_train': len(vispeech_train_dataset),
        'vimd_train': len(vimd_train_dataset),
        'seed': config.SEED
    }
)

print(f"Wandb run initialized: {wandb.run.name}")
print(f"Track at: {wandb.run.url}")

# Save training config to file as backup
config_dict = wandb.config.as_dict()
with open(f'{config.LOG_DIR}/training_config.json', 'w') as f:
    json.dump(config_dict, f, indent=2)

print(f"Training config saved to: {config.LOG_DIR}/training_config.json")

# Initialize trainer (Wandb logging is automatic)
trainer = MultiTaskTrainer(
    model=model,
    args=training_args,
    train_dataset=combined_train_dataset,
    eval_dataset=combined_val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping],
)

print(f"\nStarting training...")
print(f"  Epochs: {config.NUM_EPOCHS}")
print(f"  Steps per epoch: ~{len(combined_train_dataset) // config.BATCH_SIZE}")
print(f"  Total steps: ~{(len(combined_train_dataset) // config.BATCH_SIZE) * config.NUM_EPOCHS}")

# Train
train_result = trainer.train()

# Log final training metrics to Wandb
wandb.log({
    'final/train_loss': train_result.training_loss,
    'final/train_runtime': train_result.metrics['train_runtime'],
    'final/train_samples_per_second': train_result.metrics['train_samples_per_second']
})

# Save final training metrics to file as backup
final_metrics = {
    'train_loss': float(train_result.training_loss),
    'train_runtime': float(train_result.metrics['train_runtime']),
    'train_samples_per_second': float(train_result.metrics['train_samples_per_second'])
}

with open(f'{config.LOG_DIR}/final_train_metrics.json', 'w') as f:
    json.dump(final_metrics, f, indent=2)

print("\nTraining completed!")
print(f"Final metrics saved to: {config.LOG_DIR}/final_train_metrics.json")


TRAINING


Wandb run initialized: combined-training-42
Track at: https://wandb.ai/vuthanhlam848-vnpost/vietnamese-speaker-profiling/runs/ia5duf07
Training config saved to: /kaggle/working/logs/training_config.json

Starting training...
  Epochs: 5
  Steps per epoch: ~692
  Total steps: ~3460


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Gender Acc,Gender F1,Dialect Acc,Dialect F1,Combined F1
1,2.716500,2.034827,0.779788,0.782697,0.804029,0.792405,0.787551
2,2.033000,1.775537,0.891431,0.892059,0.806419,0.804766,0.848413
3,1.743000,1.619355,0.943667,0.943479,0.819392,0.818469,0.880974
4,1.540000,1.379736,0.950836,0.950628,0.852168,0.851085,0.900856
5,1.473000,1.428137,0.960055,0.960031,0.840901,0.840487,0.900259



Training completed!
Final metrics saved to: /kaggle/working/logs/final_train_metrics.json


In [ ]:
# 13. EVALUATION

def evaluate_and_report(trainer, dataset, dataset_name):
    """Comprehensive evaluation with file logging"""
    # Ensure log directory exists
    os.makedirs(config.LOG_DIR, exist_ok=True)
    
    print(f"\n{'='*80}")
    print(f"EVALUATING ON {dataset_name.upper()}")
    print('='*80)
    
    results = trainer.predict(dataset)
    gender_logits, dialect_logits = results.predictions
    gender_labels, dialect_labels = results.label_ids
    
    gender_preds = np.argmax(gender_logits, axis=-1)
    dialect_preds = np.argmax(dialect_logits, axis=-1)
    
    # Overall metrics
    gender_acc = accuracy_score(gender_labels, gender_preds) * 100
    dialect_acc = accuracy_score(dialect_labels, dialect_preds) * 100
    gender_f1 = f1_score(gender_labels, gender_preds, average='weighted') * 100
    dialect_f1 = f1_score(dialect_labels, dialect_preds, average='weighted') * 100
    
    print(f"\nOverall Metrics:")
    print(f"  Gender  → Accuracy: {gender_acc:.2f}%  |  F1: {gender_f1:.2f}%")
    print(f"  Dialect → Accuracy: {dialect_acc:.2f}%  |  F1: {dialect_f1:.2f}%")
    
    # Log metrics to Wandb
    wandb.log({
        f'{dataset_name}/gender_accuracy': gender_acc,
        f'{dataset_name}/gender_f1': gender_f1,
        f'{dataset_name}/dialect_accuracy': dialect_acc,
        f'{dataset_name}/dialect_f1': dialect_f1,
    })
    
    # Save metrics to JSON as backup
    metrics = {
        'gender_accuracy': gender_acc,
        'gender_f1': gender_f1,
        'dialect_accuracy': dialect_acc,
        'dialect_f1': dialect_f1,
    }
    
    with open(f'{config.LOG_DIR}/{dataset_name}_metrics.json', 'w') as f:
        json.dump(metrics, f, indent=2)
    
    # Classification reports
    print(f"\nGender Classification Report:")
    gender_report = classification_report(gender_labels, gender_preds,
                                         target_names=['Male', 'Female'],
                                         digits=4)
    print(gender_report)
    
    print(f"\nDialect Classification Report:")
    dialect_report = classification_report(dialect_labels, dialect_preds,
                                          target_names=['North', 'Central', 'South'],
                                          digits=4)
    print(dialect_report)
    
    # Save reports to text files
    with open(f'{config.LOG_DIR}/{dataset_name}_gender_report.txt', 'w') as f:
        f.write(gender_report)
    with open(f'{config.LOG_DIR}/{dataset_name}_dialect_report.txt', 'w') as f:
        f.write(dialect_report)
    
    # Confusion matrices
    gender_cm = confusion_matrix(gender_labels, gender_preds)
    dialect_cm = confusion_matrix(dialect_labels, dialect_preds)
    
    print(f"\nGender Confusion Matrix:")
    print(gender_cm)
    
    print(f"\nDialect Confusion Matrix:")
    print(dialect_cm)
    
    # Log confusion matrices to Wandb
    wandb.log({
        f'{dataset_name}/gender_confusion_matrix': wandb.plot.confusion_matrix(
            probs=None,
            y_true=gender_labels,
            preds=gender_preds,
            class_names=['Male', 'Female']
        ),
        f'{dataset_name}/dialect_confusion_matrix': wandb.plot.confusion_matrix(
            probs=None,
            y_true=dialect_labels,
            preds=dialect_preds,
            class_names=['North', 'Central', 'South']
        )
    })
    
    return {
        'gender_acc': gender_acc,
        'gender_f1': gender_f1,
        'dialect_acc': dialect_acc,
        'dialect_f1': dialect_f1
    }

# Evaluate on test sets
test_results = evaluate_and_report(trainer, combined_test_dataset, "combined_test")

# Also evaluate on individual datasets
vimd_results = evaluate_and_report(trainer, vimd_test_dataset, "vimd_test")
vispeech_results = evaluate_and_report(trainer, vispeech_test_dataset, "vispeech_test")

# Create summary table for Wandb
summary_data = [
    ['Combined', test_results['gender_acc'], test_results['dialect_acc']],
    ['ViMD', vimd_results['gender_acc'], vimd_results['dialect_acc']],
    ['ViSpeech', vispeech_results['gender_acc'], vispeech_results['dialect_acc']],
]
wandb.log({
    "test_results_summary": wandb.Table(data=summary_data, columns=["Dataset", "Gender Acc (%)", "Dialect Acc (%)"])
})

print(f"\n✓ All evaluation results saved to: {config.LOG_DIR}/")

In [ ]:
# 14. SAVE MODEL

print("\n" + "="*80)
print("SAVING MODEL")
print("="*80)

# Ensure directories exist
os.makedirs(config.LOG_DIR, exist_ok=True)
output_dir = config.OUTPUT_DIR + '/best_model'
os.makedirs(output_dir, exist_ok=True)

trainer.save_model(output_dir)
feature_extractor.save_pretrained(output_dir)

print(f"  ✓ Model saved to: {output_dir}")

# Save model info
model_info = {
    'output_dir': output_dir,
    'model_name': config.MODEL_NAME,
    'total_params': sum(p.numel() for p in model.parameters()),
    'trainable_params': sum(p.numel() for p in model.parameters() if p.requires_grad),
}

with open(f'{config.LOG_DIR}/model_info.json', 'w') as f:
    json.dump(model_info, f, indent=2)

print(f"  ✓ Model info saved to: {config.LOG_DIR}/model_info.json")

# Log model artifact to Wandb
model_artifact = wandb.Artifact(
    name=f'vietnamese-speaker-model-{wandb.run.id}',
    type='model',
    description='Multi-task speaker profiling model (gender + dialect)'
)
model_artifact.add_dir(output_dir)
wandb.log_artifact(model_artifact)

print(f"  ✓ Model logged to Wandb")

# Finish Wandb run
wandb.finish()

print("\n" + "="*80)
print("PIPELINE COMPLETED SUCCESSFULLY!")
print("="*80)
print(f"\n  View results on Wandb:")
print(f"   {wandb.run.url if wandb.run else 'https://wandb.ai'}")
print(f"\n  Logs and model also saved locally:")
print(f"   - Logs:  {config.LOG_DIR}/")
print(f"   - Model: {output_dir}/")
print("\n  Files created:")
print(f"   - training_config.json       (training parameters)")
print(f"   - final_train_metrics.json   (final training results)")
print(f"   - *_metrics.json             (evaluation metrics)")
print(f"   - *_report.txt               (classification reports)")
print(f"   - *_cm.npy                   (confusion matrices)")
print(f"   - model_info.json            (model information)")
print("\n" + "="*80)

In [ ]:
# 15. SUMMARY STATISTICS

print("\n" + "="*80)
print("FINAL SUMMARY")
print("="*80)

print(f"\nDataset Statistics:")
print(f"  Total Training Samples:   {len(combined_train_dataset):,}")
print(f"    - ViSpeech:             {len(vispeech_train_dataset):,}")
print(f"    - ViMD:                 {len(vimd_train_dataset):,}")
print(f"  Total Test Samples:       {len(combined_test_dataset):,}")
print(f"    - ViSpeech:             {len(vispeech_test_dataset):,}")
print(f"    - ViMD:                 {len(vimd_test_dataset):,}")

print(f"\nTest Results (Combined):")
print(f"  Gender Accuracy:          {test_results['gender_acc']:.2f}%")
print(f"  Dialect Accuracy:         {test_results['dialect_acc']:.2f}%")

print(f"\nTest Results (ViSpeech Only):")
print(f"  Gender Accuracy:          {vispeech_results['gender_acc']:.2f}%")
print(f"  Dialect Accuracy:         {vispeech_results['dialect_acc']:.2f}%")

print(f"\nTest Results (ViMD Only):")
print(f"  Gender Accuracy:          {vimd_results['gender_acc']:.2f}%")
print(f"  Dialect Accuracy:         {vimd_results['dialect_acc']:.2f}%")

print("\n" + "="*80)